In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly_express as px
from dash import Dash,Input,Output,dcc,html
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [2]:
df=pd.read_csv('data/processed/patient_EDR.csv')

In [3]:
option_list=list(df['neighbourhood'].unique())
option_list.insert(0,'All')
options = [{'label': 'All Neighborhoods', 'value': 'All'}]
options.extend([{'label': neighborhood, 'value': neighborhood} for neighborhood in df['neighbourhood'].unique()])
external_stylesheets = [
    {
        "href": (
            "https://fonts.googleapis.com/css2?"
            "family=Lato:wght@400;700&display=swap"
        ),
        "rel": "stylesheet",
    },
    '/assets/style.css',
    "https://stackpath.bootstrapcdn.com/bootstrap/4.5.2/css/bootstrap.min.css"

]
image_path = 'https://th.bing.com/th/id/OIG2.LAGfyagnJH3mNnlOskoe?pid=ImgGn'

app =Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(
    children=[
        html.Div([
            html.H1('Medical Appointment NO Show Analytics', className='header-title'),
            html.P('analyze a dataset of 100,000 medical appointments in Brazil, focusing on whether or not patients show up for their appointments. The dataset includes several characteristics about the patient in each row.', className='header-description'),
        ], className='header'),
        html.Div(className='row',children=[
            html.Div(className='user-control',children=[
                html.Img(src=image_path,alt='dash_image',className='Dash_image'),
                html.P('Neighbourhood',className='neighbourhood-label'),
                dcc.Dropdown(className='drop_neighbourhood',id='drop',
                    options=options,
                    value='All',
                    clearable=False),
                html.P('Gender',className='gender_label'),
                dcc.Checklist(
                        id='gender-checklist',
                        options=[{'label': 'Male', 'value': 'M'},
                                {'label': 'Female', 'value': 'F'}],
                        value=['F'],className='gender_checklist')
                ]),
                html.Div(className='figure',children=[
                    html.Div(className='pie_figure', children=[
                        html.Div([
                            dcc.Graph(id='gender_pie',className='gender_pie')],className='div_gender_pie'),
                        html.Div([
                            dcc.Graph(id='no_show_pie',className='no_show_pie')],className='div_no_show_pie'),
                        html.Div([
                            dcc.Graph(id='sms_received_pie',className='sms_received_pie')],className='div_sms_received'),
                    ]),
                    html.Div(className='chart_2',children=[
                        html.Div([
                            dcc.Graph(id='month_line_plot',className='month_line_plot')],className='month_line_plot'),
                        html.Div([
                            dcc.Graph(id='neighbourhood_client',className='neighbourhood_bar')],className='neighbourhood_client'),
                    ]),
                    html.Div(className='map_scatter',children=[
                        dcc.Graph(id='map_scatter',className='map_scatter_div')
                    ])
                    
                ])
        ]),
    ])
@app.callback(
    Output('gender_pie','figure'),
    Output('no_show_pie','figure'),
    Output('sms_received_pie','figure'),
    Input('drop','value')
)
def update_pie_figure(select_neighbourhood):
    if select_neighbourhood == 'All':
        filter_data=df
    else:
        filter_data=df[df['neighbourhood']==select_neighbourhood]
    gender_count = filter_data['gender'].value_counts().reset_index(name='Count')
    gender_count['gender'].replace({'F': 'Female', 'M': 'Male'}, inplace=True)
    #pie plot for gender
    gender_fig = px.pie(gender_count, names='gender', values='Count',color_discrete_sequence=px.colors.sequential.Agsunset,
                    hole=0.3,hover_data={'gender': True, 'Count': True})
    gender_fig.update_traces(textinfo='percent+label', pull=[0.05, 0],hoverinfo='label+percent+value',hovertemplate='<b>Count:</b> %{customdata[0]}<br>')
    gender_fig.update_layout(showlegend=False,plot_bgcolor='rgba(0,0,0,0)',paper_bgcolor='rgba(0,0,0,0)',)
    gender_fig.add_annotation(text='Gender',x=0.5,y=0.5,showarrow=False,font=dict(size=12,color='white'))

    # plot pie for no_show
    no_show = filter_data['no_show_Yes'].value_counts().reset_index(name='Count')
    no_show_fig = px.pie(no_show, names='no_show_Yes', values='Count',color_discrete_sequence=px.colors.sequential.Agsunset,
                    hole=0.3,hover_data={'no_show_Yes': True, 'Count': True})
    no_show_fig.update_traces(
    textinfo='percent+label', pull=[0.05, 0],hoverinfo='label+percent+value',hovertemplate='<b>Count:</b> %{customdata[0]}<br>')
    no_show_fig.update_layout(
        showlegend=False,plot_bgcolor='rgba(0,0,0,0)',paper_bgcolor='rgba(0,0,0,0)',
    )
    no_show_fig.add_annotation(
        text='No Show',x=0.5,y=0.5,showarrow=False,font=dict(size=12,color='white'))
    # plot pie for sms_received
    sms_received = filter_data['sms_received'].value_counts().reset_index(name='Count')
    sms_received['sms_received'].replace({1:True,0:False},inplace=True)
    sms_received_fig = px.pie(sms_received, names='sms_received', values='Count',color_discrete_sequence=px.colors.sequential.Agsunset,
                    hole=0.3,hover_data={'sms_received': True, 'Count': True})
    sms_received_fig.update_traces(
        textinfo='percent+label', pull=[0.05, 0],hoverinfo='label+percent+value',hovertemplate='<b>Count:</b> %{customdata[0]}<br>')
    sms_received_fig.update_layout(
        showlegend=False,plot_bgcolor='rgba(0,0,0,0)',paper_bgcolor='rgba(0,0,0,0)',)
    sms_received_fig.add_annotation(
        text='SMS\nReceived',x=0.5,y=0.5,showarrow=False,font=dict(size=8,color='white'))
    return gender_fig,no_show_fig,sms_received_fig
# call back for month line plot

@app.callback(
    Output('month_line_plot','figure'),
    Output('neighbourhood_client','figure'),
    Output('map_scatter','figure'),
    Input('gender-checklist','value'),
    Input('drop','value')
)

def update_month_line_plot(select_gender,select_neighbourhood):
    if select_neighbourhood == 'All':
        filter_data=df
    else:
        filter_data=df[df['neighbourhood']==select_neighbourhood]
    if not select_gender:
        fig_month=go.Figure()
        fig_month.update_layout(
            title="No Data Selected",xaxis_title="Month",yaxis_title="No Show Rate")
        fig_month.update_layout(hovermode='x',title_font=dict(color='white',size=20),title_x=0.5,
                        xaxis=dict(title='Month', showgrid=False,tickfont=dict(color='white'),title_font=dict(color='white')),
                        yaxis=dict(title='No Show Rate', showgrid=False,tickfont=dict(color='white'),title_font=dict(color='white')),
                        autosize=True,margin=dict(l=50, r=50, t=50, b=50),hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial"),
                        plot_bgcolor='rgba(0,0,0,0)',paper_bgcolor='rgba(0,0,0,0)',)
        neighbourhood_fig=go.Figure()
        neighbourhood_fig.update_layout(
            title="No Data Selected")
        neighbourhood_fig.update_layout(hovermode='x',title_font=dict(color='white',size=20),title_x=0.5,
                        xaxis=dict(title='Month', showgrid=False,tickfont=dict(color='white'),title_font=dict(color='white')),
                        yaxis=dict(title='No Show Rate', showgrid=False,tickfont=dict(color='white'),title_font=dict(color='white')),
                        autosize=True,margin=dict(l=50, r=50, t=50, b=50),hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial"),
                        plot_bgcolor='rgba(0,0,0,0)',paper_bgcolor='rgba(0,0,0,0)',)
    else:
        if isinstance(select_gender, str):
            select_gender = [select_gender]
        if select_gender==['F']:
            month_show_data=filter_data[filter_data['gender']=='F']
            neighbourhood_data=df[df['gender']=='F']
            map=df[df['gender']=='F']


        elif select_gender==['M']:
            month_show_data=filter_data[filter_data['gender']=='M']
            neighbourhood_data=df[df['gender']=='M']
            map=df[df['gender']=='M']
        else:
            month_show_data = filter_data
            neighbourhood_data=df
            map=df
        month_show=month_show_data.groupby(['scheduled_month','gender'])['no_show_Yes'].mean().reset_index()
        fig_month=px.line(month_show,x='scheduled_month',y='no_show_Yes',color='gender',
                line_dash='gender',markers=True,title='No Show Analysis by Gender and Month',)
        fig_month.update_traces(hovertemplate='<b>%{x}</b><br>%{y:.2f}% No Show Rate<br>%{color} Gender')
        fig_month.update_layout(showlegend=False,hovermode='x',title_font=dict(color='white',size=16),title_x=0.5,
                        xaxis=dict(title='Month', showgrid=False,tickfont=dict(color='white'),title_font=dict(color='white')),
                        yaxis=dict(title='No Show Rate', showgrid=False,tickfont=dict(color='white'),title_font=dict(color='white')),
                        autosize=True,
                        margin=dict(l=50, r=50, t=50, b=50),
                        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial"),
                        plot_bgcolor='rgba(0,0,0,0)',
                        paper_bgcolor='rgba(0,0,0,0)',)
        
        df_client_neighbourhood=neighbourhood_data.groupby('neighbourhood')['patient_id'].count().reset_index(name='Count')
        neighbourhood_fig=px.bar(df_client_neighbourhood,y='neighbourhood',x='Count',orientation='h',
           title='Number of clients for each neighborhood',
           labels={'Count':'Number of clients'},
           color_discrete_sequence=px.colors.sequential.Agsunset,
           height=400,
           width=650)
        neighbourhood_fig.update_layout(xaxis_title=None,yaxis_title=None)
        neighbourhood_fig.update_layout(title_font=dict(color='white',size=16),title_x=0.5,
                xaxis=dict(showgrid=False,tickfont=dict(color='white',)),
                yaxis=dict(showgrid=False,tickfont=dict(color='white',size=8)),
                autosize=True,
                margin=dict(l=60, r=60, t=50, b=50),
                hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial"),
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)',)
        fig_map = px.scatter_mapbox(map,
                        lat='lat', 
                        lon='lon', 
                        mapbox_style='carto-positron',
                        color='no_show_Yes',
                        size='age' ,
                        hover_name='neighbourhood', 
                        hover_data=['no_show_Yes','age_group', 'clinic_visits', 'scheduled_month'],  
                        zoom=1, 
                        center={"lat": 22.55139, "lon": -46.92111},
                        opacity=0.5, 
                        labels={'no_show_Yes':'No Show'})
        fig_map.update_traces(hovertemplate='<b>Neighbourhood:</b> %{hovertext}<br>'
                                '<b>No Show Yes:</b> %{customdata[0]}<br>'
                                '<b>Age Group:</b> %{customdata[1]}<br>'
                                '<b>Clinic Visits:</b> %{customdata[2]}<br>'
                                '<b>Scheduled Month:</b> %{customdata[3]}')
        fig_map.update_layout(
                            font=dict(family='Arial', size=14, color='black'),
                            hovermode='closest', 
                            autosize=True,
                            margin=dict(l=0, r=0, t=30, b=0),
                            coloraxis_showscale=False,
                            plot_bgcolor='rgba(0,0,0,0)',
                            paper_bgcolor='rgba(0,0,0,0)')  
                    
    return fig_month,neighbourhood_fig,fig_map

app.run_server(port=8051,jupyter_mode='external',debug=True)

Dash app running on http://127.0.0.1:8051/
